# The Classification of Pulsar Stars - Project Report
**By Oliver Gullery, Chan Le, Simon Lin, and Adam Parolin**

### Introduction

Pulsar stars are a rare type of neutron star that produce detectable radio emissions. Pulsars rotate and emit beams of electromagnetic radiation, which can be detected if they align directly with Earth. 

These beams take the form of radio waves - electromagnetic waves oscillating at specific frequencies that can be detected.

Using scientific equipment, we can scan for radio waves and discover new pulsar stars. However, some positive detections are caused by radio frequency interference, which makes real detections difficult to find. The main objective of our data analysis is determining if scientific equipment analyzed a real pulsar star or radio frequency interference.<br/> 

<img src="https://media.giphy.com/media/l3dj5M4YLaFww31V6/giphy.gif" width = "600"/>

Source: https://media.giphy.com/media/l3dj5M4YLaFww31V6/giphy.gif

This leads into our question: 
__Using pulsar star candidate data recorded by scientific equipment, is a given candidate a true pulsar star or just radio frequency interference?__

Each observation in the data set (the <a href="https://archive.ics.uci.edu/ml/datasets/HTRU2">HTRU2 Data Set</a> by Rob Lyon) is a candidate, with 8 continuous variables:<br />
1. `mean_of_int_profiles` <br/>
2. `sd_of_int_profiles`<br />
3. `excess_kurtosis_of_int_profiles`<br />
4. `skewness_of_int_profiles`<br />
5. `mean_of_curve`<br />
6. `sd_of_curve`<br />
7. `excess_kurtosis_of_curve`<br />
8. `skewness_of_curve`<br />

... and one class variable:<br />
1. `true_pulsar`

The first four of our predictor variables are simple statistics obtained from the integrated pulse profile. Pulse profiles are beams radiated by pulsars which have unique signatures that differ from pulsar to pulsar. 

The remaining four predictor variables are similar statistics obtained from the DM-SNR curve.  DM-SNR stands for Dispersion Measure/Signal-to-Noise Ratio, which analyzes the signal to noise ratio of pulsar beams in a normalized curve. 


### Method and Results

In [1]:
# Importing required libraries

library(tidyverse)
library(repr)
library(tidymodels)
install.packages("themis")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

We can download the dataset (https://archive.ics.uci.edu/ml/datasets/HTRU2) and import into JupyterHub.

In [2]:
# Downloading from url 
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00372/HTRU2.zip"
dir.create("data")
file <- download.file(url, "data/HTRU2.zip")

# Because the data we need is in a .zip file, we use the unzip() function in order to access "HTRU_2.csv"
star_data <- read_csv(unzip("data/HTRU2.zip", files = "HTRU_2.csv", exdir = "data/"), 
            col_names = c("mean_of_int_profiles", "sd_of_int_profiles", "excess_kurtosis_of_int_profiles",
            "skewness_of_int_profiles", "mean_of_curve", "sd_of_curve", 
            "excess_kurtosis_of_curve", "skewness_of_curve", "true_pulsar")) |>
    mutate(true_pulsar = as_factor(true_pulsar))
slice(star_data, 1:10)
# Below is a snapshot of the star data we will be working with

Warning message in dir.create("data"):
“'data' already exists”
Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): mean_of_int_profiles, sd_of_int_profiles, excess_kurtosis_of_int_pr...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


mean_of_int_profiles,sd_of_int_profiles,excess_kurtosis_of_int_profiles,skewness_of_int_profiles,mean_of_curve,sd_of_curve,excess_kurtosis_of_curve,skewness_of_curve,true_pulsar
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
140.56250,55.68378,-0.23457141,-0.6996484,3.1998328,19.110426,7.975532,74.24222,0
102.50781,58.88243,0.46531815,-0.5150879,1.6772575,14.860146,10.576487,127.39358,0
103.01562,39.34165,0.32332837,1.0511644,3.1212375,21.744669,7.735822,63.17191,0
136.75000,57.17845,-0.06841464,-0.6362384,3.6429766,20.959280,6.896499,53.59366,0
88.72656,40.67223,0.60086608,1.1234917,1.1789298,11.468720,14.269573,252.56731,0
93.57031,46.69811,0.53190485,0.4167211,1.6362876,14.545074,10.621748,131.39400,0
119.48438,48.76506,0.03146022,-0.1121676,0.9991639,9.279612,19.206230,479.75657,0
130.38281,39.84406,-0.15832276,0.3895404,1.2207358,14.378941,13.539456,198.23646,0
107.25000,52.62708,0.45268802,0.1703474,2.3319398,14.486853,9.001004,107.97251,0


#### Preliminary Exploratory Data Analysis

Before we begin classification, we will first take a closer look at our training data. For now, our intended predictor variables are going to be **mean_of_int_profiles** and **mean_of_curve**, but we will decide on our predictor variables later by using forward selection.

In [3]:
# Summarizing data into table counting the number of true and false pulsars
pulsar_frequency <- star_data |>
    group_by(true_pulsar) |>
    summarize(number = n())
pulsar_frequency

true_pulsar,number
<fct>,<int>
0,16259
1,1639


Here, we can see that there is a class imbalance - there are about 10 times more false pulsars than true pulsars! To solve this, we oversample the true pulsars in order to ensure that our classification is not skewed towards predicting false.

In [4]:
# Create upsampling recipe
ups_recipe <- recipe(true_pulsar ~ ., data = star_data) |>
  themis::step_upsample(true_pulsar, over_ratio = 1, skip = FALSE) |>
  prep()

ups_recipe

Recipe

Inputs:

      role #variables
   outcome          1
 predictor          8

Training data contained 17898 data points and no missing data.

Operations:

Up-sampling based on true_pulsar [trained]

In [5]:
# Actually using recipe to create upsampled data set
upsampled_stars <- bake(ups_recipe, star_data)

upsampled_stars |>
  group_by(true_pulsar) |>
  summarize(number = n())

true_pulsar,number
<fct>,<int>
0,16259
1,16259


Now, we can explore our upsampled data further. Let's find the average values of our intended predictor variables for both true and false pulsars.

In [6]:
# Creating another table that shows the average of our intended predictor variables
pulsar_predictors <- upsampled_stars |>
    group_by(true_pulsar) |>
    summarize(avg_mean_of_int_profiles = mean(mean_of_int_profiles),
              avg_mean_of_curve = mean(mean_of_curve))

pulsar_predictors

true_pulsar,avg_mean_of_int_profiles,avg_mean_of_curve
<fct>,<dbl>,<dbl>
0,116.56273,8.863258
1,56.72163,49.388385


From this, we note that true pulsars have a mean of integrated profile around 57 and a mean of curve around 50, while false pulsars have a mean of integrated profile around 117 and a mean of curve around 9.

#### Preminary Visualization

We will now perform some visualization to get a sense of how the data looks.

In [7]:
# Next, we will begin some visualization of our training data. We start by building scatter plots with Mean of Integrated Profiles on the x-axis 
# and Mean of DM-SNR Curve on the y-axis, with True Pulsars labeled by colour. We will build one with the original star data, and one with the 
# upsampled data.

# DO NOT REMOVE
set.seed(9999) 

# Visualizing the mean of integrated profiles and the mean of DM-SNR curve with colour to differentiate true pulsars
options(repr.plot.width = 12, repr.plot.height = 6)

pulsar_plot_ <- star_data |>
    ggplot(aes(x = mean_of_int_profiles, y = mean_of_curve)) +
           geom_point(aes(color = true_pulsar), alpha = 0.6) +
           labs(x = "Mean of Integrated Profiles", y = "Mean of DM-SNR Curve", color = "Class") +
           scale_color_discrete(labels=c('Radio Frequency Interference', 'True Pulsar')) + 
           theme(text = element_text(size = 20)) +
           ggtitle("Scatter plot for original star data")

pulsar_plot_

pulsar_plot_upsampled <- upsampled_stars |>
    ggplot(aes(x = mean_of_int_profiles, y = mean_of_curve)) +
           geom_point(aes(color = true_pulsar), alpha = 0.6) +
           labs(x = "Mean of Integrated Profiles", y = "Mean of DM-SNR Curve", color = "Class") +
           scale_color_discrete(labels=c('Radio Frequency Interference', 'True Pulsar')) + 
           theme(text = element_text(size = 18)) +
           ggtitle("Scatter plot for upsampled star data")

pulsar_plot_upsampled

Although the plots appear similar, we can see that for the upsampled plot, True Pulsars are more prominent near the bottom instead of being overshadowed by radio frequency interference.

#### Predictor variable selection

For our project we will use forward selection to select predictor variables to be used in the model. We start with an empty model and iteratively add variables that give the best performance to the model.


In [8]:

names <- colnames(star_data |> select(-true_pulsar))

formula <- paste("true_pulsar", "~", paste(names, collapse="+"))
formula

[1] "true_pulsar ~ mean_of_int_profiles+sd_of_int_profiles+excess_kurtosis_of_int_profiles+skewness_of_int_profiles+mean_of_curve+sd_of_curve+excess_kurtosis_of_curve+skewness_of_curve"

In [ ]:
# create an empty tibble to store the results
accuracies <- tibble(size = integer(), 
                     model_string = character(), 
                     accuracy = numeric())

# create a model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
     set_engine("kknn") |>
     set_mode("classification")

# create a 5-fold cross-validation object
star_vfold <- vfold_cv(star_data, v = 5, strata = true_pulsar)

# store the total number of predictors
n_total <- length(names)

# stores selected predictors
selected <- c()

# for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        # create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("true_pulsar", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        star_recipe <- recipe(as.formula(model_string), 
                                data = star_data) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors())

        # tune the KNN classifier with these predictors, 
        # and collect the accuracy for the best K
        acc <- workflow() |>
          add_recipe(star_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = star_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        # add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies |> 
      add_row(size = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}
accuracies

#### Performing the classification

The first step is to split our data into a training and a testing set:

In [ ]:
# DO NOT REMOVE
set.seed(9999) 

# Splitting data into training and testing, with true_pulsar as the strata
pulsar_split <- initial_split(upsampled_stars, prop = 0.75, strata = true_pulsar)  
pulsar_train <- training(pulsar_split)   
pulsar_test <- testing(pulsar_split)

We begin by creating a 5-fold cross validation to find the optimal *k* value:

In [ ]:
pulsar_vfold <- vfold_cv(pulsar_train, v = 5, strata = true_pulsar)

In [ ]:
# Creating a model and recipe for our tuning classification:
pulsar_recipe <- recipe(true_pulsar ~., data = pulsar_train) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors()) 

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |> 
    set_mode("classification") 

In [ ]:
# creating intervals of k values to test
k_vals <- tibble(neighbors = seq(from = 1, to = 10, by = 1))

# Combining everything into a workflow
knn_results <- workflow() |>
  add_recipe(pulsar_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = pulsar_vfold, grid = k_vals) |>
  collect_metrics() 

knn_results

In [ ]:
# Now, we will be analyzing the accuracies at different k values.
accuracies <- knn_results |> 
       filter(.metric == "accuracy")

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate")
accuracy_versus_k

In [ ]:
pulsar_training_model <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |> 
    set_engine("kknn") |> 
    set_mode("classification") # training model with k = 3 neighbors (found through 5-fold cross validation)

pulsar_fit <- workflow() |>
    add_recipe(pulsar_recipe) |>
    add_model(pulsar_training_model) |>
    fit(data = pulsar_train)

pulsar_test_predictions <- predict(pulsar_fit, pulsar_test) |> 
    bind_cols(pulsar_test) # prediction set

pulsar_prediction_accuracy <- pulsar_testpredictions |>
    metrics(trush = true_pulsar, estimate = .pred_calss) # accuracy of predictions

pulsar_prediction_accuracy

pulsar_mat <- pulsar_test_predictions |>
    conf_mat(truth = true_pulsar, estimate = .pred_class)

pulsar_mat # confusion matrix which helps us find that 4,362 of our 4,475 of our observations were labelled correctly 

### Discussion

__What did we find?__

__Is this what we expected to find?__

__What are the impacts of this finding?__
With the ability to confidently classify whether a star is pulsar or not, scientists could make enourmous strides in astronomy. By weeding out the fake pulsars and focusing only on the real ones, scientists can more accurately study what is happening inside real pulsars. Since neutron stars (which pulsars are) are the densest material in the universe, studying them can lead to enourmous discoveries related to the physics of the universe. Also, by weeding out the fake pulsars, scientists can focus in on the real pulsars and calculate cosmic distances.  This is because pulsar stars blink at regular intervals rivaled only by atomic clocks (https://www.space.com/32661-pulsars.html).  Overall, the ability to accurately predict whether a "pulsar event" in data is real or just interference is a major step in studying the universe beyond our own planet.

__What future questions are there?__
One big question is can this be applied to universes outside our own? If there are pulsar stars in our universe, there are likely stars like this in others.  Would the same algorithm for finding these stars apply outside our own universe?

Also, we must question if there are more accurate predictor variables that would work better? This also brings up, can we tune equipement in the field to also track in real time whether a reading is real or interference and filter it right away so scientists don't need to study it?

Given that this is just a predictive algorithm to determine if it is pulsar or interference, we must question the validity of the data and if the scientists behind it accurately classified the pulsars.

### References